In [12]:
#from pycocotools.coco import COCO
import gluoncv.utils as gcv
import torch
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import warnings
from torch.optim.lr_scheduler import _LRScheduler, CosineAnnealingLR
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [3]:
batch_size = 6
learning_rate = 3e-4
num_epochs = 100
seed = 41
seed_everything(seed)

In [4]:
IMAGENET_DEFAULT_MEAN = [x * 255 for x in (0.485, 0.456, 0.406)]
IMAGENET_DEFAULT_STD = [x * 255 for x in (0.229, 0.224, 0.225)]

class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''
    def __init__(self, annotation, data_dir, transforms):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image, boxes, labels = self.load_image_and_boxes(index)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        
        # Multi Scale
        target['img_size'] = torch.tensor([(512, 512)])
        target['img_scale'] = torch.tensor([1.])
        
        if self.transforms:
            # Transform 적용 후 Box가 없다면 있을 때 까지 반복 (default = 10)
            for i in range(10):
                sample = {
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': target['labels']
                }
                sample = self.transforms(**sample)
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    target['labels'] = torch.tensor(sample['labels'])
                    break
        
        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())
    
    def load_image_and_boxes(self, index):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = (image - IMAGENET_DEFAULT_MEAN) / IMAGENET_DEFAULT_STD

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])
        
        labels = np.array([x['category_id'] for x in anns])
        #labels = torch.as_tensor(labels, dtype=torch.int64)

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes, labels

In [5]:
def collate_fn(batch):
    return tuple(zip(*batch))

def get_train_transform():
    return A.Compose([
        A.RandomResizedCrop(384, 384, p=0.3),
        A.Resize(512, 512),
        A.ShiftScaleRotate(rotate_limit=30,p=0.4),
        A.HorizontalFlip(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

def get_valid_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

train_annotation = './input/data/train_all.json'
val_annotation = './input/data/val.json'
data_dir = './input/data'
train_dataset = CustomDataset(train_annotation, data_dir, get_train_transform())
val_dataset = CustomDataset(val_annotation, data_dir, get_valid_transform())

train_data_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,num_workers=4)
val_data_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False,collate_fn=collate_fn,num_workers=4)

loading annotations into memory...
Done (t=4.36s)
creating index...
index created!
loading annotations into memory...
Done (t=0.77s)
creating index...
index created!


In [6]:
def get_model(check_dir = None):
    # get weight
    # url : https://github.com/rwightman/efficientdet-pytorch
    config = get_efficientdet_config('tf_efficientdet_d6')
    config.image_size = (512, 512)
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    net = EfficientDet(config, pretrained_backbone=True)
    if check_dir != None:
        checkpoint = torch.load(check_dir)
        net.load_state_dict(checkpoint)

    net.reset_head(num_classes=11)
    net.class_net = HeadNet(config, num_outputs=11)
    return DetBenchTrain(net, config)

In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, path=None):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.path = path

    def __call__(self, val_mAP, model):
        score = val_mAP
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        model.eval()
        torch.save(model.state_dict(), self.path)
        print("model is saved..")

In [8]:
def train_fn(model, train_data_loader, val_data_loader, optimizer, scheduler, device):
    model.train()
    train_loss = AverageMeter()
    train_cls_loss = AverageMeter()
    train_box_loss = AverageMeter()
    save_path = './saved/checkpoint.pth'
    early_stop = EarlyStopping(path=save_path)
    for epoch in range(num_epochs):
        train_loss.reset()
        train_cls_loss.reset()
        train_box_loss.reset()
        model.train()
        for images, targets, image_ids in tqdm(train_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()
            current_batch_size = images.shape[0]

            targets2 = {}
            targets2['bbox'] = [target['boxes'].to(device).float() for target in targets] # variable number of instances, so the entire structure can be forced to tensor
            targets2['cls'] = [target['labels'].to(device).float() for target in targets]
            targets2['image_id'] = torch.tensor([target['image_id'] for target in targets]).to(device).float()
            targets2['img_scale'] = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
            targets2['img_size'] = torch.tensor([(512, 512) for target in targets]).to(device).float()

            # calculate loss
            losses, cls_loss, box_loss = model(images, targets2).values()

            train_loss.update(losses.detach().item(), current_batch_size)
            train_cls_loss.update(cls_loss.detach().item(), current_batch_size)
            train_box_loss.update(box_loss.detach().item(), current_batch_size)

            # backward
            optimizer.zero_grad()
            losses.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10.0)
            optimizer.step()
        scheduler.step()
        
        # Evaluation
        val_loss, val_mAP = vali_fn(val_data_loader,model,device)
        
        print(f"\nEpoch #{epoch+1} train loss: [{train_loss.avg:.4f}] train cls loss : [{train_cls_loss.avg:.4f}] train box loss : [{train_box_loss.avg:.4f}] validation mAP : [{val_mAP:.4f}] ")
        print(f"learning rate :{scheduler.get_lr()} \n")
        wandb.log({"Train Loss":train_loss.avg, "Validation mAP@50":val_mAP, "LR":scheduler.get_lr()})
        early_stop(val_mAP,model)
        if early_stop.early_stop:
            print('Stop Training.....')
            break

In [9]:
def vali_fn(val_data_loader, model, device):
    model.eval()
    vali_loss = AverageMeter()
    vali_mAP = AverageMeter()
    # Custom
    metric = gcv.metrics.VOCMApMetric(iou_thresh=0.5)
    with torch.no_grad():
        for images, targets, image_ids in tqdm(val_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()

            current_batch_size = images.shape[0]

            targets2 = {}
            targets2['bbox'] = [target['boxes'].to(device).float() for target in targets] # variable number of instances, so the entire structure can be forced to tensor
            targets2['cls'] = [target['labels'].to(device).float() for target in targets]
            targets2['image_id'] = torch.tensor([target['image_id'] for target in targets]).to(device).float()
            targets2['img_scale'] = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
            targets2['img_size'] = torch.tensor([(512, 512) for target in targets]).to(device).float()

            outputs = model(images, targets2)

            loss = outputs['loss']
            det = outputs['detections']

            # Calc Metric
            for i in range(0, len(det)):
                pred_scores=det[i, :, 4].cpu().unsqueeze_(0).numpy()
                condition=(pred_scores > 0.05)[0]
                gt_boxes=targets2['bbox'][i].cpu().unsqueeze_(0).numpy()[...,[1,0,3,2]] #move to PASCAL VOC from yxyx format
                gt_labels=targets2['cls'][i].cpu().unsqueeze_(0).numpy()

                pred_bboxes=det[i, :, 0:4].cpu().unsqueeze_(0).numpy()[:, condition, :]
                pred_labels=det[i, :, 5].cpu().unsqueeze_(0).numpy()[:, condition]
                pred_scores=pred_scores[:, condition]
                metric.update(
                  pred_bboxes=pred_bboxes,
                  pred_labels=pred_labels,
                  pred_scores=pred_scores,
                  gt_bboxes=gt_boxes,
                  gt_labels=gt_labels)

            vali_mAP.update(metric.get()[1], current_batch_size)
            vali_loss.update(loss.detach().item(), current_batch_size)
    
    # validation loss
    return vali_loss.avg, vali_mAP.avg

In [10]:
# Model
model = get_model()
model.to(device)
model.eval()
# setting
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=learning_rate)
scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-4)

In [ ]:
import wandb
wandb.init(project="object_detection")
wandb.run.name = 'effidet_alld_aug_b6'
train_fn(model, train_data_loader, val_data_loader, optimizer, scheduler, device)

wandb: Currently logged in as: jaeyeolso (use `wandb login --relogin` to force relogin)


100%|██████████| 110/110 [00:56<00:00,  1.93it/s]



Epoch #1 train loss: [42.6484] train cls loss : [42.0276] train box loss : [0.0124] validation mAP : [0.0421] 
learning rate :[0.0002975452469668926] 



  0%|          | 0/546 [00:00<?, ?it/s]

model is saved..


...

  0%|          | 0/546 [00:00<?, ?it/s]

model is saved..


  0%|          | 0/546 [00:00<?, ?it/s]


Epoch #23 train loss: [0.4445] train cls loss : [0.2859] train box loss : [0.0032] validation mAP : [0.6074] 
learning rate :[0.00012427203147438512] 



 11%|█         | 59/546 [01:13<10:04,  1.24s/it]

In [21]:
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-4)
train_fn(model, train_data_loader, val_data_loader, optimizer, scheduler, device)

100%|██████████| 55/55 [00:30<00:00,  1.81it/s]



Epoch #1 train loss: [0.3044] train cls loss : [0.1998] train box loss : [0.0021] validation mAP : [0.2210]
learning rate :[0.00029033107648888907] 



  0%|          | 0/219 [00:00<?, ?it/s]

model is saved..


100%|██████████| 55/55 [00:30<00:00,  1.79it/s]


...


Epoch #13 train loss: [0.1273] train cls loss : [0.0805] train box loss : [0.0009] validation mAP : [0.2946]
learning rate :[0.00018897178975242352] 



  0%|          | 0/219 [00:00<?, ?it/s]


Epoch #14 train loss: [0.1389] train cls loss : [0.0884] train box loss : [0.0010] validation mAP : [0.2589]
learning rate :[0.0002158273731636108] 



 15%|█▌        | 33/219 [00:47<04:30,  1.45s/it]


KeyboardInterrupt: 

In [13]:
import wandb

wandb.init(project="object_detection")
wandb.run.name = 'effidet_cosine'
train_fn(model, train_data_loader, val_data_loader, optimizer, scheduler, device)

100%|██████████| 55/55 [00:32<00:00,  1.68it/s]



Epoch #1 train loss: [141.78404961590735] train cls loss : [140.95246401291382] train box loss : [0.01663171035694197] validation mAP : [0.0002717303096907903] 



  0%|          | 0/219 [00:00<?, ?it/s]

model is saved..


100%|██████████| 55/55 [00:32<00:00,  1.71it/s]


...


Epoch #31 train loss: [0.2884030363055919] train cls loss : [0.1958930447532514] train box loss : [0.0018501998293492907] validation mAP : [0.2111629326965684] 



  0%|          | 0/219 [00:00<?, ?it/s]


Epoch #32 train loss: [0.28559418432030187] train cls loss : [0.1922134641681776] train box loss : [0.0018676144012559973] validation mAP : [0.21342555627809118] 



  6%|▌         | 13/219 [00:19<05:16,  1.54s/it]


KeyboardInterrupt: 